In [1]:
import pandas as pd
import psycopg2 
from io import StringIO

## ABRINDO OS CSV IMPORTADOS DA INTERNET


In [2]:
df = pd.read_csv("/home/gabriel/Downloads/dados_abertos_rios___limpos(1).csv", sep=';')

In [3]:
df2 = pd.read_csv("/home/gabriel/Downloads/cnuc_2022_01sem.csv", encoding_errors='ignore', on_bad_lines='skip', sep=';')

In [4]:
df = df[
['Data do mutirão',
 'Horário de início',
 'Ambiente ',
 'Estado',
 'Cidade',
 'Nome do local (exemplo: praia de boa viagem)',
 'Bairro ou distrito',
 'Instituição Organizadora',
 'Caráter da instituição',
 'Número aproximado de participantes (total)',
 'Duração total (coleta + triagem)',
 'Peso total dos resíduos (kg)',
 'Quantidade total de itens coletados',
 'Plástico (total)',
 '1. Fragmentos de plástico (maiores que 2,5 cm).',
 '2. Brinquedos',
 '3. Canudos',
 '4. Copos/talheres/pratos',
 '5. Embalagens de alimento',
 '6. Escovas de dente',
 '7. Esponjas/espumas',
 '8. Galões',
 '9. Garrafas PET',
 '10. Hastes de cotonete/pirulito',
 '11. Isqueiros',
 '12. Pedaços de isopor (maiores que 2,5 cm)',
 '13. Pinos de plástico',
 '14. Sacos e sacolas',
 '15. Tampas/lacres/argolas de garrafa',
 '16. Outros (descreva)',
 'Borracha (total)',
 '1. Balões e bexigas',
 '2. Câmaras de ar',
 '3. Chinelos/sandálias',
 '4. Luvas',
 '5. Pneus',
 '6. Preservativos (camisinha)',
 '7. Outros (descreva)',
 'Madeira (total)',
 '1. Espetos/varas',
 '2. Fósforos',
 '3. Palitos de dente',
 '4. Palitos de sorvete/picolé',
 '5. Pedaços de madeira',
 '6. Outros (descreva)',
 'Produtos têxteis (total)',
 '1. Aviamentos (linhas/barbantes)',
 '2. Calçados (sapatos, tênis)',
 '3. Roupas e pedaços de tecido',
 '4. Outros (descreva)',
 'Papel (total)',
 '1. Caixas de leite/suco, etc',
 '2. Embalagens de cigarro',
 '3. Embalagens de papel',
 '4. Jornais/panfletos/revistas/livros',
 '5. Papelão',
 '6. Pedaços de papel/guardanapos',
 '7. Outros (Descreva)2',
 'Metal (total)',
 '1. Anéis de lacre de latas de bebidas',
 '2. Latas de bebidas',
 '3. Latas (comida, desodorante, óleo, tinta, etc.)',
 '4. Pedaços de metal',
 '5. Tampas de metal',
 '6. Outros (descreva)2',
 'Vidro (total)',
 '1. Copos/xícaras/pratos',
 '2. Garrafas',
 '3. Lâmpadas',
 '4. Pedaços de cerâmica',
 '5. Pedaços de vidro',
 '6. Potes de vidro ou cerâmica',
 '7. Outros (descreva)3',
 'Materiais de Pesca (total)',
 '1. Anzóis',
 '2. Boias',
 '3. Linhas de pesca',
 '4. Pedaços de corda',
 '5. Redes/pedaços de rede',
 '6. Outros (descreva)5',
 'Outros Resíduos (total)',
 '1. Bitucas/guimbas/filtros de cigarro',
 '2. Cera de vela/parafina',
 '3. Eletroeletrônicos (TV, celular, computador, etc.)',
 '4. Entulhos/material de construção',
 '5. Fraldas e absorventes',
 '6. Seringas',
 '7. Outros (descreva)6',
 'Animais',
 'Animais Vivos',
 'Animais Mortos']
]

df.loc[0, 'Instituição Organizadora'] = 'Greenpeace Poa'

In [5]:
df2 = df2.rename(columns={
    'Cdigo UC':'Codigo UC',
    'Informaes Gerais':'Informacoes Gerais',
    'Ano de Criao':'Ano de Criacao',
    'rgo Gestor':'Orgao Gestor',
    'rea soma biomas':'Area soma biomas',
    'rea soma Biomas Continental':'Area soma biomas continentais',
    'Bioma rea (ha)':'Bioma area (ha)',
    'Amaznia':'Amazonia',
    'Mata Atlntica':'Mata Atlantica',
    'rea Marinha':'Area Marinha',
    'Municpio Costeiro':'Municipio Costeiro',
    'Municpio Costeiro + rea Marinha':'Municipio Costeiro + Area Marinha',
    'Municpios Abrangidos': 'Cidade', 'UF': 'Estado'
})

df2 = df2[
['ID_UC',
 'Codigo UC',
 'Informacoes Gerais',
 'Nome da UC',
 'Esfera Administrativa',
 'Categoria de Manejo',
 'Categoria IUCN',
 'Estado',
 'Ano de Criacao',
 'Cidade',
 'Plano de Manejo',
 'Conselho Gestor',
 'Orgao Gestor',
 'Area soma biomas',
 'Area soma biomas continentais',
 'Area Marinha',
 'Bioma declarado',
 'Grupo',
 'Mar Territorial',
 'Municipio Costeiro',
 'Municipio Costeiro + Area Marinha',
 'Reserva da Biosfera']
]

df2['Cidade'] = df2['Cidade'].str.split(', ')
df2 = df2.explode('Cidade')
df2['Cidade'] = df2['Cidade'].str.replace(r'\([A-Z]{2}\)','', regex=True).str.strip()
df2['Estado'] = df2['Estado'].str.split(', ')
df2 = df2.explode('Estado')


## SEPARANDO OS DATAFRAMES QUE SE TORNARÃO TABELAS DO BANCO

In [6]:
df_dados_coleta = df[[
'1. Fragmentos de plástico (maiores que 2,5 cm).',
 '2. Brinquedos',
 '3. Canudos',
 '4. Copos/talheres/pratos',
 '5. Embalagens de alimento',
 '6. Escovas de dente',
 '7. Esponjas/espumas',
 '8. Galões',
 '9. Garrafas PET',
 '10. Hastes de cotonete/pirulito',
 '11. Isqueiros',
 '12. Pedaços de isopor (maiores que 2,5 cm)',
 '13. Pinos de plástico',
 '14. Sacos e sacolas',
 '15. Tampas/lacres/argolas de garrafa',
 '1. Balões e bexigas',
 '2. Câmaras de ar',
 '3. Chinelos/sandálias',
 '4. Luvas',
 '5. Pneus',
 '6. Preservativos (camisinha)',
 '1. Espetos/varas',
 '2. Fósforos',
 '3. Palitos de dente',
 '4. Palitos de sorvete/picolé',
 '5. Pedaços de madeira',
 '1. Aviamentos (linhas/barbantes)',
 '2. Calçados (sapatos, tênis)',
 '3. Roupas e pedaços de tecido',
 '1. Caixas de leite/suco, etc',
 '2. Embalagens de cigarro',
 '3. Embalagens de papel',
 '4. Jornais/panfletos/revistas/livros',
 '5. Papelão',
 '6. Pedaços de papel/guardanapos',
 '1. Anéis de lacre de latas de bebidas',
 '2. Latas de bebidas',
 '3. Latas (comida, desodorante, óleo, tinta, etc.)',
 '4. Pedaços de metal',
 '5. Tampas de metal',
 '1. Copos/xícaras/pratos',
 '2. Garrafas',
 '3. Lâmpadas',
 '4. Pedaços de cerâmica',
 '5. Pedaços de vidro',
 '6. Potes de vidro ou cerâmica',
 '1. Anzóis',
 '2. Boias',
 '3. Linhas de pesca',
 '4. Pedaços de corda',
 '5. Redes/pedaços de rede',
 '1. Bitucas/guimbas/filtros de cigarro',
 '2. Cera de vela/parafina',
 '3. Eletroeletrônicos (TV, celular, computador, etc.)',
 '4. Entulhos/material de construção',
 '5. Fraldas e absorventes',
 '6. Seringas',
 'Animais',
 'Animais Vivos',
 'Animais Mortos'
]]
df_dados_coleta['id_dados_coleta'] = range(1, len(df_dados_coleta)+1)
df_dados_coleta = df_dados_coleta.fillna(0)
df_dados_coleta = df_dados_coleta[[
 'id_dados_coleta',
 '1. Fragmentos de plástico (maiores que 2,5 cm).',
 '3. Canudos',
 '5. Embalagens de alimento',
 '9. Garrafas PET',
 '14. Sacos e sacolas',
 '5. Pneus',
 '6. Preservativos (camisinha)',
 '2. Embalagens de cigarro',
 '4. Jornais/panfletos/revistas/livros',
 '3. Latas (comida, desodorante, óleo, tinta, etc.)',
 '4. Entulhos/material de construção',
 'Animais Mortos'
]]

df_dados_coleta = df_dados_coleta.rename(columns={
 '1. Fragmentos de plástico (maiores que 2,5 cm).':'Fragmentos de plástico',
 '3. Canudos': 'Canudos',
 '5. Embalagens de alimento': 'Embalagens de alimento',
 '9. Garrafas PET':'Garrafas PET',
 '14. Sacos e sacolas':'Sacolas',
 '5. Pneus':'Pneus',
 '6. Preservativos (camisinha)':'Preservativos',
 '2. Embalagens de cigarro':'Embalagens de cigarro',
 '4. Jornais/panfletos/revistas/livros':'Jornais/panfletos/revistas/livros',
 '3. Latas (comida, desodorante, óleo, tinta, etc.)':'Latas',
 '4. Entulhos/material de construção':'Entulhos',
})

/tmp/ipykernel_8347/1996394059.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dados_coleta['id_dados_coleta'] = range(1, len(df_dados_coleta)+1)


In [7]:
df_instituicao_organizadora = df[[
     'Instituição Organizadora',
     'Caráter da instituição'
]]
df_instituicao_organizadora = df_instituicao_organizadora.drop_duplicates('Instituição Organizadora').reset_index(drop=True)
df_instituicao_organizadora['id_instituicao_organizadora'] = range(1, len(df_instituicao_organizadora)+1)
df_instituicao_organizadora = df_instituicao_organizadora[[
    'id_instituicao_organizadora',
    'Instituição Organizadora',
    'Caráter da instituição'
]]

In [8]:
df_municipio = pd.concat([df[['Cidade', 'Estado']],df2[['Cidade', 'Estado']]]).drop_duplicates().reset_index(drop=True)


df_municipio['Cidade'] = df_municipio['Cidade'].str.replace(r'\([A-Z]{2}\)','', regex=True).str.strip()  

df_municipio = df_municipio.drop_duplicates('Cidade').reset_index()
df_municipio['id_municipio'] = range(1, len(df_municipio) + 1)
df_municipio = df_municipio[[
    'id_municipio',
    'Cidade', 'Estado']]

In [9]:
df_mutirao = df[[
    'Data do mutirão',
    'Horário de início',
    'Número aproximado de participantes (total)',
    'Duração total (coleta + triagem)',
    'Cidade', 'Instituição Organizadora'
]]
df_mutirao['id_mutirao'] = range(1, len(df_mutirao)+1)
df_merge = pd.merge(df_mutirao, df_municipio, on='Cidade', how='inner')
df_merge2 = pd.merge(df_merge, df_instituicao_organizadora, on='Instituição Organizadora', how='inner')
df_mutirao = df_merge2[[
 'id_mutirao',
 'Data do mutirão',
 'Horário de início',
 'Número aproximado de participantes (total)',
 'Duração total (coleta + triagem)',
 'id_municipio',
 'id_instituicao_organizadora']]

/tmp/ipykernel_8347/3612770269.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mutirao['id_mutirao'] = range(1, len(df_mutirao)+1)


In [10]:
df_local_coleta = df[[
     'Nome do local (exemplo: praia de boa viagem)'
]]
df_local_coleta['id_local_coleta'] = range(1, len(df_local_coleta)+1)
df_local_coleta = df_local_coleta[[
     'id_local_coleta',
     'Nome do local (exemplo: praia de boa viagem)'
]]

/tmp/ipykernel_8347/2672952102.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_local_coleta['id_local_coleta'] = range(1, len(df_local_coleta)+1)


In [11]:
df_coleta_lixo = df[[
     'Plástico (total)',
    'Borracha (total)',
    'Madeira (total)',
    'Produtos têxteis (total)',
    'Papel (total)',
    'Metal (total)',
    'Vidro (total)',
    'Materiais de Pesca (total)',
    'Outros Resíduos (total)'
]]
df_coleta_lixo['id_coleta_lixo'] = range(1, len(df_coleta_lixo)+1)
df_coleta_lixo = df_coleta_lixo[[
    'id_coleta_lixo',
    'Plástico (total)',
    'Borracha (total)',
    'Madeira (total)',
    'Produtos têxteis (total)',
    'Papel (total)',
    'Metal (total)',
    'Vidro (total)',
    'Materiais de Pesca (total)',
    'Outros Resíduos (total)'
]]

/tmp/ipykernel_8347/2714029214.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_coleta_lixo['id_coleta_lixo'] = range(1, len(df_coleta_lixo)+1)


In [12]:
df_esfera = df2[['Esfera Administrativa']]
df_esfera = df_esfera.drop_duplicates('Esfera Administrativa').reset_index(drop=True)
df_esfera['id_esfera'] = range(1, len(df_esfera)+1)
df_esfera = df_esfera[['id_esfera','Esfera Administrativa']]

In [13]:
df_manejo = df2[[
    'Categoria de Manejo',
     'Categoria IUCN',
    'Plano de Manejo'
]]
df_manejo = df_manejo.drop_duplicates('Categoria de Manejo').reset_index(drop=True)
df_manejo['id_manejo'] = range(1, len(df_manejo)+1)
df_manejo = df_manejo[[
    'id_manejo',
    'Categoria de Manejo',
     'Categoria IUCN',
    'Plano de Manejo'
]]

In [14]:
df_bioma = df2[['Bioma declarado']]
df_bioma = df_bioma.drop_duplicates('Bioma declarado').reset_index(drop=True)
df_bioma['id_bioma'] = range(1, len(df_bioma)+1)
df_bioma = df_bioma[['id_bioma', 'Bioma declarado']]

In [15]:
df_uc = df2[[
 'Nome da UC',
 'Ano de Criacao',
 'Esfera Administrativa',
 'Area soma biomas',
 'Area soma biomas continentais',
 'Area Marinha',
 'Bioma declarado',
 'Cidade',
 'Categoria de Manejo'
]]
df_uc = df_uc.drop_duplicates('Nome da UC')
df_uc['id_uc'] = range(1, len(df_uc)+1)
df_a = pd.merge(df_uc, df_bioma, on='Bioma declarado', how='inner')
df_b = pd.merge(df_a, df_municipio, on='Cidade', how='inner')
df_c = pd.merge(df_b, df_manejo, on='Categoria de Manejo', how='inner')
df_d = pd.merge(df_c, df_esfera, on='Esfera Administrativa', how='inner')
df_uc = df_d[[
 'id_uc',
 'Nome da UC',
 'Ano de Criacao',
 'Area soma biomas',
 'Area soma biomas continentais',
 'Area Marinha',
 'id_bioma',
 'id_municipio',
 'id_manejo',
 'id_esfera']]

df_uc['Area soma biomas'] = pd.to_numeric(df_uc['Area soma biomas'], errors='coerce').astype('float64')
df_uc['Area soma biomas continentais'] = pd.to_numeric(df_uc['Area soma biomas continentais'], errors='coerce').astype('float64')
df_uc['Area Marinha'] = pd.to_numeric(df_uc['Area Marinha'], errors='coerce').astype('float64')

/tmp/ipykernel_8347/4233489840.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_uc['Area soma biomas'] = pd.to_numeric(df_uc['Area soma biomas'], errors='coerce').astype('float64')
/tmp/ipykernel_8347/4233489840.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_uc['Area soma biomas continentais'] = pd.to_numeric(df_uc['Area soma biomas continentais'], errors='coerce').astype('float64')
/tmp/ipykernel_8347/4233489840.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

## IMPORTANDO SQL E CRIANDO O CURSOR PARA AS QUERIES

In [16]:
from psycopg2 import sql
conn = psycopg2.connect(dbname = "", user = "postgres", password = "", host = "localhost", port = "5432")
cursor = conn.cursor()

## CRIANDO AS TABELAS NO BANCO

In [ ]:
sql_criacao = '''
DROP TABLE IF EXISTS unidade_de_conservacao CASCADE;
DROP TABLE IF EXISTS categoria_manejo CASCADE;
DROP TABLE IF EXISTS bioma CASCADE;
DROP TABLE IF EXISTS tipo_de_lixo CASCADE;
DROP TABLE IF EXISTS local_de_coleta CASCADE;
DROP TABLE IF EXISTS instituicao_organizadora CASCADE;
DROP TABLE IF EXISTS municipio CASCADE;
DROP TABLE IF EXISTS esfera_administrativa CASCADE;
DROP TABLE IF EXISTS mutirao_de_limpeza CASCADE;
DROP TABLE IF EXISTS coleta_de_lixo CASCADE;


CREATE TABLE IF NOT EXISTS categoria_manejo (
    id_manejo INT PRIMARY KEY,
    categoria_de_manejo TEXT NOT NULL,
    categoria_iucn TEXT NOT NULL,
    plano_de_manejo TEXT NOT NULL
);

CREATE TABLE IF NOT EXISTS bioma (
    id_bioma INT PRIMARY KEY,
    bioma_declarado TEXT NOT NULL
);

CREATE TABLE IF NOT EXISTS municipio (
    id_municipio INT PRIMARY KEY,
    cidade TEXT NOT NULL,
    estado TEXT NOT NULL
);

CREATE TABLE IF NOT EXISTS esfera_administrativa (
    id_esfera INT PRIMARY KEY,
    esfera_administrativa TEXT NOT NULL
);

CREATE TABLE IF NOT EXISTS unidade_de_conservacao (
    id_uc INT PRIMARY KEY,
    nome_da_uc TEXT NOT NULL,
    ano_de_criacao INT,
    area_soma_biomas FLOAT,
    area_soma_biomas_continentais FLOAT,
    area_marinha FLOAT,
    id_bioma INT,
    id_municipio INT,
    id_manejo INT,
    id_esfera INT,
    FOREIGN KEY (id_manejo) REFERENCES categoria_manejo(id_manejo),
    FOREIGN KEY (id_bioma) REFERENCES bioma(id_bioma),
    FOREIGN KEY (id_municipio) REFERENCES municipio(id_municipio),
    FOREIGN KEY (id_esfera) REFERENCES esfera_administrativa(id_esfera)
);

CREATE TABLE IF NOT EXISTS instituicao_organizadora (
    id_instituicao_organizadora INT PRIMARY KEY,
    instituicao_organizadora TEXT NOT NULL,
    carater_da_instituicao TEXT NOT NULL
);

CREATE TABLE IF NOT EXISTS mutirao_de_limpeza (
    id_mutirao INT PRIMARY KEY,
    data_do_mutirao DATE NOT NULL,
    horario_de_inicio TIME NOT NULL,
    numero_aproximado_de_participantes INT,
    duracao_total TIME NOT NULL,
    id_municipio INT,
    id_instituicao_organizadora INT,
    FOREIGN KEY (id_municipio) REFERENCES municipio(id_municipio),
    FOREIGN KEY (id_instituicao_organizadora) REFERENCES instituicao_organizadora(id_instituicao_organizadora)
);

CREATE TABLE IF NOT EXISTS local_de_coleta (
    id_mutirao INT PRIMARY KEY,
    nome_do_local TEXT NOT NULL,
    FOREIGN KEY (id_mutirao) REFERENCES mutirao_de_limpeza(id_mutirao)
);

CREATE TABLE IF NOT EXISTS coleta_de_lixo (
    id_mutirao INT PRIMARY KEY,
    plastico INT,
    borracha INT,
    madeira INT,
    produtos_texteis INT,
    papel INT,
    metal INT,
    vidro INT,
    materiais_de_pesca INT,
    outros_residuos INT,
    FOREIGN KEY (id_mutirao) REFERENCES mutirao_de_limpeza(id_mutirao)
);

CREATE TABLE IF NOT EXISTS tipo_de_lixo (
    id_mutirao INT PRIMARY KEY,
    fragmentos_de_plastico INT,
    canudos INT,
    embalagens_de_alimento INT,
    garrafas_pet INT,
    sacolas INT, 
    pneus INT,
    preservativos INT,
    embalagens_de_cigarro INT,
    jornais_panfletos_revistas_livros INT,
    latas INT, entulhos INT, animais_mortos INT,
    FOREIGN KEY (id_mutirao) REFERENCES coleta_de_lixo(id_mutirao)
);
'''

cursor.execute(sql_criacao)

In [ ]:
for indice, linha in df_municipio.iterrows():
    cursor.execute(
        sql.SQL("""
            INSERT INTO municipio (id_municipio, cidade, estado)
            VALUES (%s, %s, %s)
        """),
        (linha['id_municipio'], linha['Cidade'], linha['Estado'])
    )

for indice, linha in df_instituicao_organizadora.iterrows():
    cursor.execute(
        sql.SQL("""
            INSERT INTO instituicao_organizadora (id_instituicao_organizadora, instituicao_organizadora, carater_da_instituicao)
            VALUES (%s, %s, %s)
        """),
        (linha['id_instituicao_organizadora'], linha['Instituição Organizadora'], linha['Caráter da instituição'])
    )

for indice, linha in df_mutirao.iterrows():
    cursor.execute(
        sql.SQL("""
            INSERT INTO mutirao_de_limpeza (id_mutirao, data_do_mutirao, horario_de_inicio, numero_aproximado_de_participantes,
            duracao_total, id_municipio, id_instituicao_organizadora)
            VALUES (%s, %s, %s, %s, %s, %s, %s)
        """),  
        (linha['id_mutirao'], linha['Data do mutirão'], linha['Horário de início'], linha['Número aproximado de participantes (total)'],
         linha['Duração total (coleta + triagem)'],linha['id_municipio'],linha['id_instituicao_organizadora'])
    )

for indice, linha in df_local_coleta.iterrows():
    cursor.execute(
        sql.SQL("""
            INSERT INTO local_de_coleta (id_mutirao, nome_do_local)
            VALUES (%s, %s)
        """),
        (linha['id_local_coleta'], linha['Nome do local (exemplo: praia de boa viagem)'])
    )

for indice, linha in df_esfera.iterrows():
    cursor.execute(
        sql.SQL("""
            INSERT INTO esfera_administrativa (id_esfera, esfera_administrativa)
            VALUES (%s, %s)
        """),
        (linha['id_esfera'], linha['Esfera Administrativa'])
    )

for indice, linha in df_coleta_lixo.iterrows():
    cursor.execute(
        sql.SQL("""
            INSERT INTO coleta_de_lixo (id_mutirao, plastico, borracha,
            madeira, produtos_texteis, papel, metal, vidro, materiais_de_pesca, outros_residuos)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """),
(int(linha['id_coleta_lixo']), int(linha['Plástico (total)']), int(linha['Borracha (total)']), int(linha['Madeira (total)']),
 int(linha['Produtos têxteis (total)']), int(linha['Papel (total)']), int(linha['Metal (total)']), int(linha['Vidro (total)']),
 int(linha['Materiais de Pesca (total)']), int(linha['Outros Resíduos (total)']))
    )

for indice, linha in df_manejo.iterrows():
    cursor.execute(
        sql.SQL("""
            INSERT INTO categoria_manejo (id_manejo, categoria_de_manejo, categoria_iucn, plano_de_manejo)
            VALUES (%s, %s, %s, %s)
        """),
        (linha['id_manejo'], linha['Categoria de Manejo'], linha['Categoria IUCN'], linha['Plano de Manejo'])
    )

for indice, linha in df_bioma.iterrows():
    cursor.execute(
        sql.SQL("""
            INSERT INTO bioma (id_bioma, bioma_declarado)
            VALUES (%s, %s)
        """),
        (linha['id_bioma'], linha['Bioma declarado'])
    )

for indice, linha in df_uc.iterrows():
    cursor.execute(
        sql.SQL("""
            INSERT INTO unidade_de_conservacao (id_uc, nome_da_uc, ano_de_criacao, area_soma_biomas, area_soma_biomas_continentais, area_marinha,
            id_bioma, id_municipio, id_manejo, id_esfera)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """),
        (linha['id_uc'], linha['Nome da UC'], linha['Ano de Criacao'], linha['Area soma biomas'], linha['Area soma biomas continentais'], linha['Area Marinha'], 
         linha['id_bioma'], linha['id_municipio'], linha['id_manejo'], linha['id_esfera'])
    )

for indice, linha in df_dados_coleta.iterrows():
    cursor.execute(
        sql.SQL("""
            INSERT INTO tipo_de_lixo (id_mutirao, fragmentos_de_plastico, canudos, embalagens_de_alimento, garrafas_pet, sacolas, pneus,
            preservativos, embalagens_de_cigarro, jornais_panfletos_revistas_livros, latas, entulhos, animais_mortos)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """),
(int(linha['id_dados_coleta']), int(linha['Fragmentos de plástico']), int(linha['Canudos']), int(linha['Embalagens de alimento']),
 int(linha['Garrafas PET']), int(linha['Sacolas']), int(linha['Pneus']), int(linha['Preservativos']), int(linha['Embalagens de cigarro']),
 int(linha['Jornais/panfletos/revistas/livros']), int(linha['Latas']), int(linha['Entulhos']), int(linha['Animais Mortos']))
    )

conn.commit()

## QUERY 1  -  NESSA QUERY ANALISAREMOS A PROPORÇÃO DE UNIDADES DE CONSERVAÇÃO AMBIENTAL E DE LIXO EXTRAÍDO EM CADA CIDADE EM QUE OCORREU UM MUTIRÃO DE LIMPEZA

In [ ]:
query_1 = '''
    SELECT
    	cidade, COUNT(nome_da_uc) AS qtd_uc, SUM(lixo_total) AS lixo_total_municipio
    FROM (
    	SELECT
    		"municipio".cidade,
    		"unidade_de_conservacao".nome_da_uc,
    		"mutirao_de_limpeza".id_municipio,
    		"mutirao_de_limpeza".id_mutirao,
    		(plastico + borracha + madeira + produtos_texteis + papel + 
    		 metal + vidro + materiais_de_pesca + outros_residuos) AS lixo_total
    	FROM
    		"coleta_de_lixo"
    	INNER JOIN "mutirao_de_limpeza" ON "coleta_de_lixo".id_mutirao = "mutirao_de_limpeza".id_mutirao
    	LEFT JOIN "unidade_de_conservacao" ON "mutirao_de_limpeza".id_municipio = "unidade_de_conservacao".id_municipio
    	INNER JOIN "municipio" ON "mutirao_de_limpeza".id_municipio = "municipio".id_municipio
    ) AS new_query
    GROUP BY cidade
    ORDER BY qtd_uc
'''
cursor.execute(query_1)
resultado = cursor.fetchall()
colunas = [desc[0] for desc in cursor.description]
query_1 = pd.DataFrame(resultado, columns=colunas)

## QUERY 2 - NESSA QUERY O OBJETIVO É ANALISAR A PROPORÇÃO DE LIXO EXTRAÍDO NOS ESTADOS QUE HOUVE MUTIRÕES, ANALISANDO A PROPORÇÃO TAMBÉM COM A QUANTIDADE DE UNIDADES DE CONSERVAÇÃO PRESENTES NELE

In [ ]:
query_2 = '''
    SELECT
    	estado, COUNT(nome_da_uc) AS qtd_uc, SUM(lixo_total) AS lixo_total_estado
    FROM (
    	SELECT
    		"coleta_de_lixo".id_mutirao,
    		"municipio".estado,
    		"unidade_de_conservacao".nome_da_uc,
    		"mutirao_de_limpeza".id_municipio,
    		"mutirao_de_limpeza".id_mutirao,
    		(plastico + borracha + madeira + produtos_texteis + papel + 
    		 metal + vidro + materiais_de_pesca + outros_residuos) AS lixo_total
    	FROM
    		"coleta_de_lixo"
    	INNER JOIN "mutirao_de_limpeza" ON "coleta_de_lixo".id_mutirao = "mutirao_de_limpeza".id_mutirao
    	LEFT JOIN "unidade_de_conservacao" ON "mutirao_de_limpeza".id_municipio = "unidade_de_conservacao".id_municipio
    	INNER JOIN "municipio" ON "mutirao_de_limpeza".id_municipio = "municipio".id_municipio
    ) AS new_query
    GROUP BY estado
    ORDER BY lixo_total_estado;
'''
cursor.execute(query_2)
resultado = cursor.fetchall()
colunas = [desc[0] for desc in cursor.description]
query_2 = pd.DataFrame(resultado, columns=colunas)

## QUERY 3 - NESSA QUERY ANALISAREMOS O TIPO E A QUANTIDADE DE LIXO EXTRAÍDO PELOS MUTIRÕES NOS BIOMAS BRASILEIROS EM QUE ELAS FORAM PRESENTES

In [31]:
query_3 = '''
    SELECT (plastico_total + canudo_total + embalagens_de_alimento_total + garrafas_pet_total + sacolas_total + pneus_total +
	   preservativos_total + embalagens_de_cigarro_total + jornais_panfletos_revistas_livros_total +
	   latas_total + entulhos_total) AS qtd_lixo_total, *
    FROM(
    	SELECT b.bioma_declarado, COUNT(mcp.cidade) AS cidades_total, SUM(fragmentos_de_plastico) AS plastico_total, SUM(canudos) AS canudo_total,
    	       SUM(embalagens_de_alimento) AS embalagens_de_alimento_total, SUM(garrafas_pet) AS garrafas_pet_total,
    		   SUM(sacolas) AS sacolas_total, SUM(pneus) AS pneus_total, SUM(preservativos) AS preservativos_total, SUM(embalagens_de_cigarro) AS embalagens_de_cigarro_total,
    		   SUM(jornais_panfletos_revistas_livros) AS jornais_panfletos_revistas_livros_total, SUM(latas) AS latas_total, SUM(entulhos) AS entulhos_total
    	FROM "tipo_de_lixo" AS t
    	JOIN "mutirao_de_limpeza" AS m ON m.id_mutirao = t.id_mutirao
    	JOIN "unidade_de_conservacao" AS u ON u.id_municipio = m.id_municipio
    	JOIN "municipio" AS mcp ON mcp.id_municipio = m.id_municipio
    	JOIN "bioma" AS b ON b.id_bioma = u.id_bioma
    	GROUP BY bioma_declarado
    )
ORDER BY qtd_lixo_total
'''
cursor.execute(query_3)
resultado = cursor.fetchall()
colunas = [desc[0] for desc in cursor.description]
query_3 = pd.DataFrame(resultado, columns=colunas)

## QUERY 4 - NESSA ANALISAREMOS A EFICIÊNCIA DA INSTITUIÇÃO ORGANIZADORA PELO CARÁTER (PÚBLICO, INICIATIVA PRIVADA, ETC), POR BASE NA QUANTIDADE DE LIXO EXTRAÍDA POR ELAS

In [ ]:
query_4 = '''
    SELECT carater_da_instituicao, COUNT(id_mutirao) AS qtd_mutiroes, SUM(lixo_total) AS lixo_total
    FROM(
    	SELECT
    		"instituicao_organizadora".carater_da_instituicao,
    		"municipio".cidade,
    		"mutirao_de_limpeza".id_municipio,
    		"mutirao_de_limpeza".id_mutirao,
    		(plastico + borracha + madeira + produtos_texteis + papel + 
    		 metal + vidro + materiais_de_pesca + outros_residuos) AS lixo_total
    	FROM
    		"coleta_de_lixo"
    	INNER JOIN "mutirao_de_limpeza" ON "coleta_de_lixo".id_mutirao = "mutirao_de_limpeza".id_mutirao
    	INNER JOIN "municipio" ON "mutirao_de_limpeza".id_municipio = "municipio".id_municipio
    	INNER JOIN "instituicao_organizadora" ON 
    		"instituicao_organizadora".id_instituicao_organizadora = "mutirao_de_limpeza".id_instituicao_organizadora
    )
    GROUP BY carater_da_instituicao
    ORDER BY lixo_total;
'''
cursor.execute(query_4)
resultado = cursor.fetchall()
colunas = [desc[0] for desc in cursor.description]
query_4 = pd.DataFrame(resultado, columns=colunas)

## QUERY 5 - NESSA QUERY ANALISAREMOS A EFICIENCIA DOS MUTIROES EM UM ANO ESPECIFICO, COMO POR EXEMPLO, LIXO EXTRAIDO POR PESSOA EM UM ANO, MÉDIA DE LIXO TIRADO POR MINUTO EM UM ANO, QUANTO DE LIXO FOI EXTRAIDO EM UM ANO E ETC

In [ ]:
query_5 = '''
    SELECT *, (lixo_total_ano/participantes_no_ano) AS lixo_extraido_por_pessoa, (lixo_total_ano/(EXTRACT(EPOCH FROM tempo_total_de_mutirao::INTERVAL)/60))
           AS lixo_extraido_por_minuto
    FROM(
    	SELECT EXTRACT(YEAR FROM data_do_mutirao) AS ano, SUM(numero_aproximado_de_participantes) AS participantes_no_ano, 
    		   SUM(duracao_total::INTERVAL) AS tempo_total_de_mutirao, COUNT(id_municipio) AS total_municipios, SUM(plastico) AS plastico_total,
    		   SUM(borracha) AS borracha_total, SUM(madeira) AS madeira_total, SUM(produtos_texteis) AS produtos_texteis_total,
    		   SUM(papel) AS papel_total, SUM(metal) AS metal, SUM(vidro) AS vidro_total, SUM(materiais_de_pesca) AS
    	       materiais_de_pesca_total, SUM(outros_residuos) AS outros_residuos_total, SUM(plastico + borracha + madeira + produtos_texteis
    		   + papel + metal + vidro + materiais_de_pesca + outros_residuos) AS lixo_total_ano
    	FROM "mutirao_de_limpeza" AS m
    	JOIN coleta_de_lixo AS c ON c.id_mutirao = m.id_mutirao 
    	GROUP BY ano
    	ORDER BY ano
    )
'''
cursor.execute(query_5)
resultado = cursor.fetchall()
colunas = [desc[0] for desc in cursor.description]
query_5 = pd.DataFrame(resultado, columns=colunas)